In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import datetime

2023-01-08 08:51:39.587100: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-08 08:51:39.803978: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 08:51:39.804011: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-01-08 08:51:41.643831: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
df = pd.read_csv("datasets/train.csv", encoding = "ISO-8859-1", names=['polarity', 'id', 'query', 'user', 'text'], index_col=2)
df = df.sample(frac=1)[:30_000] # shuffle and truncate
df['polarity'] = df['polarity'].apply(lambda x: 1 if x == 4 else 0)

tdf, vdf = train_test_split(df, test_size=0.2)
train_data = tdf['text'].to_numpy()
train_label = tdf['polarity'].to_numpy()

val_data = vdf['text'].to_numpy()
val_label = vdf['polarity'].to_numpy()

In [3]:
VOCAB_SIZE = 200_000  # full vocab for 1.6M dataset contains 850061 tokens
encoder = tf.keras.layers.TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_data)

2023-01-08 08:51:49.788297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-08 08:51:49.788396: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-08 08:51:49.788462: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-08 08:51:49.788550: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcufft.so.10'; dlerror: libcufft.so.10: cannot open shared object file: No such file or directory
2023-01-08 08:51:49.788640: W tensorfl

In [4]:
vocab = np.array(encoder.get_vocabulary())
word_index = dict(zip(vocab, range(len(vocab))))

print(len(vocab))
vocab[:40]

37822


array(['', '[UNK]', 'i', 'to', 'the', 'a', 'my', 'and', 'you', 'is', 'it',
       'in', 'for', 'of', 'im', 'on', 'me', 'so', 'have', 'that', 'but',
       'just', 'with', 'its', 'be', 'at', 'not', 'was', 'this', 'good',
       'now', 'up', 'get', 'day', 'all', 'out', 'like', 'are', 'no', 'go'],
      dtype='<U96')

In [5]:
encoder("Hello, how's the weather today?")

<tf.Tensor: shape=(5,), dtype=int64, numpy=array([ 420, 1056,    4,  265,   49])>

In [6]:
#embedding_layer(encoder('I am'))

In [7]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        input_dim=len(encoder.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])

In [8]:
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True]


In [9]:
sample_text = ('the gig last night turned out great')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

1/1 [==============================] - 2s 2s/step
[-0.00639106]
1/1 [==============================] - 0s 183ms/step
[-0.00639106]


In [10]:
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_filepath = 'tmp/checkpoint'

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=0)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


history = model.fit(x=train_data, y=train_label, validation_data=(val_data, val_label), epochs=10, 
                    callbacks=[tensorboard_callback, model_checkpoint_callback])

Epoch 1/10
749/750 [============================>.] - ETA: 0s - loss: 0.6281 - accuracy: 0.5866

INFO:tensorflow:Assets written to: tmp/checkpoint/assets


INFO:tensorflow:Assets written to: tmp/checkpoint/assets


750/750 [==============================] - 70s 84ms/step - loss: 0.6282 - accuracy: 0.5865 - val_loss: 0.5611 - val_accuracy: 0.6848
Epoch 2/10
750/750 [==============================] - ETA: 0s - loss: 0.4589 - accuracy: 0.7849

INFO:tensorflow:Assets written to: tmp/checkpoint/assets


INFO:tensorflow:Assets written to: tmp/checkpoint/assets


750/750 [==============================] - 86s 114ms/step - loss: 0.4589 - accuracy: 0.7849 - val_loss: 0.5376 - val_accuracy: 0.7140
Epoch 3/10
750/750 [==============================] - 63s 84ms/step - loss: 0.3442 - accuracy: 0.8578 - val_loss: 0.5773 - val_accuracy: 0.7053
Epoch 4/10
750/750 [==============================] - 62s 83ms/step - loss: 0.2501 - accuracy: 0.9052 - val_loss: 0.7096 - val_accuracy: 0.6833
Epoch 5/10
750/750 [==============================] - 66s 88ms/step - loss: 0.1703 - accuracy: 0.9390 - val_loss: 0.8880 - val_accuracy: 0.6743
Epoch 6/10
750/750 [==============================] - 68s 91ms/step - loss: 0.1188 - accuracy: 0.9580 - val_loss: 1.0757 - val_accuracy: 0.6603
Epoch 7/10
332/750 [============>.................] - ETA: 39s - loss: 0.0816 - accuracy: 0.9739

KeyboardInterrupt: 

In [ ]:
model.predict(np.array(["Shit!", "Oh fuck!", "You'd better shut up!", "I wanna kill this bastard", "That was amazing", "OMG!"
                        "That was cute", "Nice one", "I loved it", "I really liked how he behaved", "He was a nice dude"]))

1/1 [==============================] - 2s 2s/step


array([[-1.5173236 ],
       [-2.7038252 ],
       [-0.32582322],
       [-3.0155642 ],
       [ 3.6182864 ],
       [ 3.0159233 ],
       [ 2.916467  ],
       [ 1.1423533 ],
       [ 1.2402942 ],
       [ 1.9487635 ]], dtype=float32)